In [12]:
import yfinance as yf, time, random # Avoir accès aux données de Yahoo finance via datareader
import pandas as pd 
from pathlib import Path #permet d'utiliser des chemins de fichiers

In [15]:
tickers = ["VFV", "XIU", "ZAG"]

#on va télécharger les prix de ces trois ETF depuis le 01.01.2015

TICKERS = ["VFV.TO", "XIU.TO", "ZAG.TO"]
START   = "2015-01-01"
prices  = pd.DataFrame()

def fetch_adj_close(tick, start, retries=6):
    #télécharger les données Yahoo avec back-off; return the ‘Close’ series (déjà ajusté).#
    
    for k in range(retries):
        df = yf.download(tick, start=start, progress=False, threads=False)  # auto_adjust=True par défaut. le prix est ajusté au dividende et au split 
        if not df.empty:
            return df["Close"]                # total-return series, si la requête réussie, on renvoie à la série close 
            
        wait = 3 * 2**k + random.random()     # 3 s, 6 s, 12 s… c'est le temps d'attente pour contourner l'erreur de multi requête au serveur de yahoo 
        print(f"{tick}: rate-limited, retry in {wait:.1f} s")
        time.sleep(wait)
    raise RuntimeError(f"Failed to fetch {tick} after {retries} retries") #Indique que si l'erreur persiste c'est qu'il n'y a pas de donnée concernant le tickers en question sur Yahoo


for t in TICKERS:
    prices[t.split(".")[0]] = fetch_adj_close(t, START) #Permet de traiter chaque tickers un à un 

#la site permet de sauvegarder mes prix 
out = Path("..") / "data" / "prices_raw.csv"
prices.to_csv(out)
print("✓ Adjusted prices saved →", out.resolve())
prices.tail()
#prices est un DataFrame (tableau) indexé par date. Les colonnes sont les tickers et les lignes les colonnes. À l'intérieur c'est les prix. 

✓ Adjusted prices saved → /Users/neil/Documents/MidLongPortfolio/data/prices_raw.csv


,VFV,XIU,ZAG
Date,,,
2025-07-08,151.070007,40.490002,13.75
2025-07-09,151.970001,40.580002,13.80
2025-07-10,152.190002,40.779999,13.77
2025-07-11,151.910004,40.650002,13.69
2025-07-14,152.360001,40.939999,13.68


In [16]:
prices.isna().sum() #s'assure qu'il n'y a pas de trou dans la série ce qui pourrait poser problème par la suite. Si 0 = pas de trous 

VFV    0
XIU    0
ZAG    0
dtype: int64